In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.0.0-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
sc=spark.sparkContext

In [4]:
import numpy as np
#np.set_printoptions(threshold=sys.maxsize)

In [98]:
datos=(
    sc.textFile('../Base/BD_100.csv')
    .map(lambda s: s.split(','))
    .map(lambda col: col[0])
    )

In [99]:
datos.count()

100

In [100]:
datos.take(10)

['COMENTARIO',
 'MEXICANOS NO DEJARAN BEBER REFRESCOS PESE AUMENTO HABRA PROXIMO ANO PERO SI ESTAN CAMBIANDO SUS GUSTOS OPCIONES LIGHT SEGUN ESTUDIO DE',
 'SEMAFORO ALERTA ENCUENTRA EN',
 'RUTH RODRIGUEZ ESCRIBE CISMA SALUD LLAMADO MOHAR',
 'MARTIN ESPINOSA ESCRIBE FUERZA ESTADOS',
 'REPORTAN DESAPARICION MAESTRO UAM IZTAPALAPA AYUDANOS COMPARTIR',
 'TERMINA TU DIA BIEN INFORMADO POR',
 'YOUTUBER ENCONTRO NUEVAMENTE AMOR I',
 'A DIEZ MESES INICIO GOBIERNO PRESIDENTE AFIRMO VAMOS BIEN',
 'CUATRO MUJERES INDIGENAS CHIAPAS GRADUARON INGENIERAS INDIA NINGUNA ELLAS HABIA SALIDO COMUNIDAD HABLAN POCO ESPANOL CONOCIMIENTOS ADQUIRIERON PODRAN AYUDAR HABITANTES SUS LUGARES ORIGEN']

In [101]:
TotalMuestras = datos.count()# Son las filas que tiene con datos
print("Total de registros original: ", TotalMuestras)#imprimir en pantalla

Total de registros original:  100


In [102]:
#Para el proceso de etiquetado solo utilizaremos la columna del encabezado y borraremos el encabezado: 
rdd_datos = (
    datos
    .zipWithIndex()
    .filter(lambda col: col[1] > 0)
    .map(lambda col: col[0])
    )

In [103]:
#RDD que almacena las palabras separadas de la base.
rdd_separadas = rdd_datos.flatMap(lambda s:s.split(' '))

In [104]:
rdd_separadas.take(10)

['MEXICANOS',
 'NO',
 'DEJARAN',
 'BEBER',
 'REFRESCOS',
 'PESE',
 'AUMENTO',
 'HABRA',
 'PROXIMO',
 'ANO']

In [105]:
#Se agrega un 1 para contar cada palabra.
rdd_cont = rdd_separadas.map(lambda s: (s,1))

In [106]:
rdd_cont.take(10)

[('MEXICANOS', 1),
 ('NO', 1),
 ('DEJARAN', 1),
 ('BEBER', 1),
 ('REFRESCOS', 1),
 ('PESE', 1),
 ('AUMENTO', 1),
 ('HABRA', 1),
 ('PROXIMO', 1),
 ('ANO', 1)]

In [107]:
#Se guarda un archivo con las palabras separadas. 
rdd_cont.coalesce(1).saveAsTextFile('../resultados/100_com/palabras_diferentes')

In [108]:
rdd_cont.count()

737

In [109]:
#Se suma cada palabra que aparece y se agrupan.
rdd_agrupadas = rdd_cont.reduceByKey(lambda a, b: a + b)

In [110]:
rdd_agrupadas.take(15)

[('MEXICANOS', 2),
 ('NO', 29),
 ('BEBER', 1),
 ('REFRESCOS', 1),
 ('AUMENTO', 2),
 ('HABRA', 3),
 ('SI', 1),
 ('ESTAN', 2),
 ('GUSTOS', 1),
 ('ESTUDIO', 1),
 ('DE', 2),
 ('RODRIGUEZ', 1),
 ('SALUD', 5),
 ('LLAMADO', 1),
 ('MOHAR', 1)]

In [111]:
#Total de Palabras diferentes.
rdd_agrupadas.count()

533

In [112]:
#Se ordenan las palabras según las veces que aparecen de mayor a menor. 
rdd_ordenadas = rdd_agrupadas.sortBy(lambda s:s[1], False)

In [113]:
rdd_ordenadas.take(10)

[('NO', 29),
 ('ESCRIBE', 17),
 ('PRESIDENTE', 12),
 ('PARA', 10),
 ('ANDRES', 9),
 ('YA', 7),
 ('SUS', 6),
 ('SALUD', 5),
 ('POR', 5),
 ('FUE', 5)]

In [120]:
#Numero total de palabras que hay en el archivo
rdd_ordenadas.count()

533

In [116]:
rdd_ordenadas.coalesce(1).saveAsTextFile('../resultados/100_com/no_palabras')

In [121]:
s = rdd_ordenadas.map(lambda x: x[0])

In [122]:
#Éste es el RDD que contiene solamente las palabras, ya órdenadas por cantidad de repetición.
s.take(5)

['NO', 'ESCRIBE', 'PRESIDENTE', 'PARA', 'ANDRES']

In [123]:
#Éste nuevo RDD sólo agrega el índice de las palabras pero, dicho índice, comienza en 0.
rdd_indice = s.zipWithIndex()

In [124]:
rdd_indice.take(10)

[('NO', 0),
 ('ESCRIBE', 1),
 ('PRESIDENTE', 2),
 ('PARA', 3),
 ('ANDRES', 4),
 ('YA', 5),
 ('SUS', 6),
 ('SALUD', 7),
 ('POR', 8),
 ('FUE', 9)]

In [126]:
#Nuevo RDD con la palabra y el índice comenzando en 1.
rdd_palabras = rdd_indice.map(lambda x: (x[0], x[1] + 1))

In [127]:
rdd_palabras.take(10)

[('NO', 1),
 ('ESCRIBE', 2),
 ('PRESIDENTE', 3),
 ('PARA', 4),
 ('ANDRES', 5),
 ('YA', 6),
 ('SUS', 7),
 ('SALUD', 8),
 ('POR', 9),
 ('FUE', 10)]

In [129]:
rdd_palabras.coalesce(1).saveAsTextFile('../resultados/100_com/indice_100')